In [1]:
# === Load full table ===
import os
import pandas as pd
import numpy as np

FULL_TABLE = "all_cities_all_ILR_energy.csv" 

full = pd.read_csv(FULL_TABLE)
need = ["City","ILR","PDC_MWp","PAC_MW"]
missing = [c for c in need if c not in full.columns]
if missing:
    raise ValueError(f"Faltan columnas en {FULL_TABLE}: {missing}")

full["ILR"] = pd.to_numeric(full["ILR"], errors="coerce")
full["PDC_MWp"] = pd.to_numeric(full["PDC_MWp"], errors="coerce")
full["PAC_MW"] = pd.to_numeric(full["PAC_MW"], errors="coerce")
full = full.sort_values(["City","ILR"]).reset_index(drop=True)

cities = full["City"].unique().tolist()
print("✔ Cargado:", FULL_TABLE, "| ciudades:", len(cities))

✔ Cargado: all_cities_all_ILR_energy.csv | ciudades: 28


In [2]:
# === Wage per countries (2023) ===
wage_rows = [
    ("Austria",        54.508),
    ("Belgium",        57.989),
    ("Bulgaria",       13.503),
    ("Croatia",        21.523),
    ("Cyprus",         26.430),
    ("Czech Republic", 23.454),
    ("Denmark",        67.604),
    ("Estonia",        24.899),
    ("Finland",        48.391),
    ("France",         42.662),
    ("Germany",        50.998),
    ("Greece",         17.013),
    ("Hungary",        16.895),
    ("Ireland",        58.679),
    ("Italy",          32.749),
    ("Latvia",         22.293),
    ("Lithuania",      27.178),
    ("Luxembourg",     81.064),
    ("Malta",          29.989),
    ("Netherlands",    55.548),
    ("Poland",         18.054),
    ("Portugal",       22.933),
    ("Romania",        17.739),
    ("Slovenia",       33.081),
    ("Slovakia",       19.001),
    ("Spain",          32.587),
    ("Sweden",         44.619),
    ("United Kingdom", 50.880),
]
wages = pd.DataFrame(wage_rows, columns=["Country", "wage_2023"])
ref_mean = wages["wage_2023"].mean()
wages["wage_factor"] = wages["wage_2023"] / ref_mean

# === City → Country ===
opt_map = pd.read_csv("optimal_ilr_by_country_all_scenarios.csv",
                      usecols=["City", "Country"]).drop_duplicates()

# === City → wage_factor ===========================
city_factor_df = opt_map.merge(wages[["Country","wage_factor"]],
                               on="Country", how="left")
missing = city_factor_df[city_factor_df["wage_factor"].isna()]["City"].tolist()
if missing:
    print("⚠️ Ciudades sin factor salarial (se aplicará 1.0):", missing)

wage_factor_by_city = dict(zip(city_factor_df["City"],
                               city_factor_df["wage_factor"].fillna(1.0)))

In [3]:
wage_factor_by_city

{'Vienna': 1.507734649987207,
 'Brussels': 1.6040218796893695,
 'Sofia': 0.37350372383461616,
 'Zagreb': 0.5953433050501699,
 'Nicosia': 0.7310748293674668,
 'Prague': 0.6487563014750121,
 'Copenhagen': 1.8699804299870688,
 'Tallinn': 0.6887261512077396,
 'Helsinki': 1.3385335629179376,
 'Paris': 1.1800648645658294,
 'Berlin': 1.410645257210824,
 'Athens': 0.47059311661099945,
 'Budapest': 0.46732914272279047,
 'Dublin': 1.6231078286966927,
 'Rome': 0.9058633971606195,
 'Riga': 0.616642117710516,
 'Vilnius': 0.7517651045232318,
 'Luxembourg': 2.242294739608185,
 'Valletta': 0.8295196011313267,
 'Amsterdam': 1.5365018774765058,
 'Warsaw': 0.49938800489596086,
 'Lisbon': 0.6343450269346999,
 'Bucharest': 0.49067485426218294,
 'Bratislava': 0.5255827783886204,
 'Ljubljana': 0.9150467813206649,
 'Madrid': 0.901382348263248,
 'Stockholm': 1.234197041677904,
 'London': 1.4073812833226151}

In [20]:
# === Corporate income tax rates by city (in decimal form) ===
tax_rows = [
    ("Austria","Vienna",0.23),
    ("Belgium","Brussels",0.25),
    ("Bulgaria","Sofia",0.10),
    ("Croatia","Zagreb",0.18),
    ("Cyprus","Nicosia",0.1250),
    ("Czech Republic","Prague",0.21),
    ("Denmark","Copenhagen",0.22),
    ("Estonia","Tallinn",0.22),
    ("Finland","Helsinki",0.20),
    ("France","Paris",0.3613),
    ("Germany","Berlin",0.3006),
    ("Greece","Athens",0.22),
    ("Hungary","Budapest",0.09),
    ("Ireland","Dublin",0.1250),
    ("Italy","Rome",0.24),
    ("Latvia","Riga",0.20),
    ("Lithuania","Vilnius",0.16),
    ("Luxembourg","Luxembourg",0.2387),
    ("Malta","Valletta",0.35),
    ("Netherlands","Amsterdam",0.258),
    ("Poland","Warsaw",0.19),
    ("Portugal","Lisbon",0.305),
    ("Romania","Bucharest",0.16),
    ("Slovakia","Bratislava",0.24),
    ("Slovenia","Ljubljana",0.22),
    ("Spain","Madrid",0.25),
    ("Sweden","Stockholm",0.206),
    ("United Kingdom","London",0.25),
]
ti_by_city = {city: ti for _, city, ti in tax_rows}

In [21]:
# === Cost parameters and functions ===

def default_cost_params():
    """
    Return the base cost parameters (identical to your previous script).
    All values in EUR.
    """
    return dict(
        module_cost_per_mwp        = 250_000,  # €/MWp
        bos_mounting_cost_per_mwp  = 125_000,  # €/MWp (mounting/BOS)
        epc_connection_per_mw      = 100_000,  # €/MW AC (substation/connection)
        inverter_cost_per_mw       = 25_000,   # €/MW AC
        access_cost_per_mw         = 100_000,  # €/MW AC (access)
        om_cost_perc               = 0.01,     # % sobre (módulo+mounting+inverter)
        imp_seg_terr               = 0.01,     # % sobre (module+BoS)
    )

def compute_city_ilr_costs_from_full(
    full_table: pd.DataFrame,
    city: str,
    cost_params: dict | None = None,
    wage_factor_by_city: dict | None = None,
) -> pd.DataFrame:
    """
    Compute costs per ILR using PDC_MWp (per row) and PAC_MW (city-level constant).

    Apply salary re-scaling only to:

    MountingCost (BOS)

    OMCost (O&M)
    """
    if cost_params is None:
        cost_params = default_cost_params()

    sub = full_table[full_table["City"].str.casefold() == city.casefold()].copy()
    if sub.empty:
        raise ValueError(f"No hay datos para la ciudad '{city}' en la tabla.")
    sub = sub.sort_values("ILR").reset_index(drop=True)

    PDC = sub["PDC_MWp"].to_numpy(dtype=float)
    PAC = float(sub["PAC_MW"].iloc[0])
    ILR_vals = sub["ILR"].round(2).to_list()

    # City factor (1.0 if missing).
    wf = (wage_factor_by_city or {}).get(city, 1.0)

    p = cost_params
    module_cost   = PDC * p["module_cost_per_mwp"]
    mounting_cost = PDC * p["bos_mounting_cost_per_mwp"] * wf  # <- wage factor correction

    df = pd.DataFrame({
        "City"                : city,
        "ILR"                 : ILR_vals,
        "DC_Nominal_Power_MW" : PDC,
        "ModuleCost"          : module_cost,
        "MountingCost"        : mounting_cost,
    })

    df["HV_ConnectionSubstationCost"] = PAC * p["epc_connection_per_mw"]
    df["InverterCost"]                = PAC * p["inverter_cost_per_mw"]
    df["AccessCost"]                  = PAC * p["access_cost_per_mw"]

    # O&M rescaled; taxes/insurance
    df["OMCost"]             = (df["ModuleCost"] + df["MountingCost"] + df["InverterCost"]) * p["om_cost_perc"] * wf
    df["TaxesInsuranceCost"] = (df["ModuleCost"] + df["MountingCost"]) * p["imp_seg_terr"]

    df["TotalCost"] = (
        df["ModuleCost"] + df["MountingCost"] + df["InverterCost"] +
        df["AccessCost"] + df["HV_ConnectionSubstationCost"]
    )

    df["ratio_NominalPower_TotalCost"] = df["TotalCost"] / df["DC_Nominal_Power_MW"]
    df["Incremental_Cost"]             = df["TotalCost"].diff()
    df["Incremental_Cost_Percentage"]  = df["Incremental_Cost"] / df["TotalCost"].shift(1)

    cols = [
        "City","ILR","DC_Nominal_Power_MW",
        "ModuleCost","MountingCost","InverterCost","AccessCost","HV_ConnectionSubstationCost",
        "OMCost","TaxesInsuranceCost",
        "TotalCost","ratio_NominalPower_TotalCost","Incremental_Cost","Incremental_Cost_Percentage",
    ]
    return df[cols]

In [22]:
# === Run for ALL cities and export results. ===
from datetime import datetime

params = default_cost_params()
all_costs = []

for c in cities:
    try:
        df_c = compute_city_ilr_costs_from_full(
            full, c, params,
            wage_factor_by_city=wage_factor_by_city
        )
        all_costs.append(df_c)
    except Exception as e:
        print(f"❌ {c}: {e}")

df_ILR_cost_euros = pd.concat(all_costs, ignore_index=True)
print("✔ Costes calculados:", df_ILR_cost_euros.shape)

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
out_csv = f"ilr_costs_by_city_{ts}.csv"
df_ILR_cost_euros.to_csv(out_csv, index=False)
print(f"💾 Exportado: {out_csv}")

df_ILR_cost_euros.head()

✔ Costes calculados: (1120, 14)
💾 Exportado: ilr_costs_by_city_2025_10_02_19_37_02.csv


,City,ILR,DC_Nominal_Power_MW,ModuleCost,MountingCost,InverterCost,AccessCost,HV_ConnectionSubstationCost,OMCost,TaxesInsuranceCost,TotalCost,ratio_NominalPower_TotalCost,Incremental_Cost,Incremental_Cost_Percentage
0,Amsterdam,1.00,50.09,12522500.0,9.620422e+06,1251250.0,5005000.0,5005000.0,359451.897844,221429.223803,3.340417e+07,666883.058102,NaN,NaN
1,Amsterdam,1.05,52.46,13115000.0,1.007561e+07,1251250.0,5005000.0,5005000.0,375549.654101,231906.110616,3.445186e+07,656726.287868,1.047689e+06,0.031364
2,Amsterdam,1.10,54.82,13705000.0,1.052888e+07,1251250.0,5005000.0,5005000.0,391579.487336,242338.791154,3.549513e+07,647485.025819,1.043268e+06,0.030282
3,Amsterdam,1.14,57.18,14295000.0,1.098215e+07,1251250.0,5005000.0,5005000.0,407609.320570,252771.471693,3.653840e+07,639006.596175,1.043268e+06,0.029392
4,Amsterdam,1.19,59.54,14885000.0,1.143542e+07,1251250.0,5005000.0,5005000.0,423639.153805,263204.152231,3.758167e+07,631200.289270,1.043268e+06,0.028553


In [24]:
# === Parameters and energy including degradation. ===
import os, glob
import numpy as np
import pandas as pd

# inputs
FULL_TABLE = "all_cities_all_ILR_energy.csv"   # generated in the previous block.
PPA_VALUES = [35, 50, 65, 100]                 # €/MWh
LIFE_YEARS  = 25
DEGR_RATE   = 0.27 / 100                       # 0.27% yearly

# Load energy data (City, ILR, AC_energy_MWh)
need_cols = ["City","ILR","AC_energy_MWh"]
full_energy = pd.read_csv(FULL_TABLE, usecols=need_cols)
full_energy["ILR"] = pd.to_numeric(full_energy["ILR"], errors="coerce")
full_energy = (full_energy
               .dropna(subset=["ILR","AC_energy_MWh"])
               .drop_duplicates(subset=["City","ILR"])
               .sort_values(["City","ILR"])
               .reset_index(drop=True))

# Add columns year1..year25 applying degradation to AC_energy_MWh.
year_cols = [f"year{i}" for i in range(1, LIFE_YEARS + 1)]
for i in range(1, LIFE_YEARS + 1):
    full_energy[f"year{i}"] = full_energy["AC_energy_MWh"] * ((1 - DEGR_RATE) ** (i - 1))

print("✔ Energía preparada:", full_energy.shape, "| columnas anuales:", len(year_cols))
full_energy.head()

✔ Energía preparada: (1120, 28) | columnas anuales: 25


,City,ILR,AC_energy_MWh,year1,year2,year3,year4,year5,year6,year7,...,year16,year17,year18,year19,year20,year21,year22,year23,year24,year25
0,Amsterdam,1.00,43289.709469,43289.709469,43172.827254,43056.260620,42940.008717,42824.070693,42708.445702,42593.132899,...,41569.227773,41456.990858,41345.056983,41233.425329,41122.095081,41011.065424,40900.335547,40789.904641,40679.771899,40569.936515
1,Amsterdam,1.05,45340.811361,45340.811361,45218.391170,45096.301514,44974.541500,44853.110238,44732.006840,44611.230422,...,43538.811833,43421.257041,43304.019647,43187.098794,43070.493627,42954.203294,42838.226945,42722.563733,42607.212811,42492.173336
2,Amsterdam,1.10,47391.195372,47391.195372,47263.239145,47135.628399,47008.362202,46881.439624,46754.859737,46628.621616,...,45507.706543,45384.835735,45262.296679,45140.088478,45018.210239,44896.661071,44775.440087,44654.546398,44533.979123,44413.737379
3,Amsterdam,1.14,49440.860070,49440.860070,49307.369747,49174.239849,49041.469402,48909.057434,48777.002979,48645.305071,...,47475.910528,47347.725569,47219.886710,47092.393016,46965.243555,46838.437397,46711.973616,46585.851288,46460.069489,46334.627302
4,Amsterdam,1.19,51487.159193,51487.159193,51348.143864,51209.503875,51071.238215,50933.345871,50795.825838,50658.677108,...,49440.882698,49307.392315,49174.262355,49041.491847,48909.079819,48777.025304,48645.327335,48513.984951,48382.997192,48252.363100


In [25]:
# === Load costs by city/ILR. ===
# If df_ILR_cost_euros is already in memory, it is used directly.
try:
    df_ILR_cost_euros
except NameError:
    # Load the most recent cost CSV if not already in memory.
    cost_files = sorted(glob.glob("ilr_costs_by_city_*.csv"))
    if not cost_files:
        raise FileNotFoundError("No encuentro 'ilr_costs_by_city_*.csv' y df_ILR_cost_euros no está en memoria.")
    latest_cost = cost_files[-1]
    df_ILR_cost_euros = pd.read_csv(latest_cost)
    print("✔ Costes cargados de:", latest_cost)

# Ensure correct data types and uniqueness of the City–ILR pair.
df_ILR_cost_euros["ILR"] = pd.to_numeric(df_ILR_cost_euros["ILR"], errors="coerce")
df_ILR_cost_euros = (df_ILR_cost_euros
                     .dropna(subset=["ILR"])
                     .drop_duplicates(subset=["City","ILR"])
                     .sort_values(["City","ILR"])
                     .reset_index(drop=True))

# Cost columns
need_cost_cols = [
    "City","ILR",
    "ModuleCost","MountingCost","InverterCost","AccessCost","HV_ConnectionSubstationCost",
    "OMCost","TaxesInsuranceCost","TotalCost"
]
missing = [c for c in need_cost_cols if c not in df_ILR_cost_euros.columns]
if missing:
    raise ValueError(f"Faltan columnas de coste en df_ILR_cost_euros: {missing}")

print("✔ Costes listos:", df_ILR_cost_euros.shape)
df_ILR_cost_euros.head()


✔ Costes listos: (1120, 14)


,City,ILR,DC_Nominal_Power_MW,ModuleCost,MountingCost,InverterCost,AccessCost,HV_ConnectionSubstationCost,OMCost,TaxesInsuranceCost,TotalCost,ratio_NominalPower_TotalCost,Incremental_Cost,Incremental_Cost_Percentage
0,Amsterdam,1.00,50.09,12522500.0,9.620422e+06,1251250.0,5005000.0,5005000.0,359451.897844,221429.223803,3.340417e+07,666883.058102,NaN,NaN
1,Amsterdam,1.05,52.46,13115000.0,1.007561e+07,1251250.0,5005000.0,5005000.0,375549.654101,231906.110616,3.445186e+07,656726.287868,1.047689e+06,0.031364
2,Amsterdam,1.10,54.82,13705000.0,1.052888e+07,1251250.0,5005000.0,5005000.0,391579.487336,242338.791154,3.549513e+07,647485.025819,1.043268e+06,0.030282
3,Amsterdam,1.14,57.18,14295000.0,1.098215e+07,1251250.0,5005000.0,5005000.0,407609.320570,252771.471693,3.653840e+07,639006.596175,1.043268e+06,0.029392
4,Amsterdam,1.19,59.54,14885000.0,1.143542e+07,1251250.0,5005000.0,5005000.0,423639.153805,263204.152231,3.758167e+07,631200.289270,1.043268e+06,0.028553


In [28]:
# === Join and compute revenues based on PPA. ===

# Unir por City + ILR (1:1)
df_joined = pd.merge(
    full_energy[["City","ILR","AC_energy_MWh"] + year_cols],
    df_ILR_cost_euros[need_cost_cols],
    on=["City","ILR"],
    how="inner"
)

def calcular_beneficio(df_join: pd.DataFrame, ppa_eur_mwh: float,
                       life_years: int = LIFE_YEARS) -> pd.DataFrame:
    """
    Return a DataFrame with cashflows per City/ILR:
    year0 = –CAPEX (TotalCost)

    year1..yearN = Revenues (Energy * PPA) – (OMCost + TaxesInsuranceCost)
    """
    out = df_join[["City","ILR"]].copy()

    # Initial investment (Uses totalcost)
    inversion_inicial = df_join["TotalCost"]
    out.insert(2, "year0", -inversion_inicial.values)

    # years 1..N
    om_tax = (df_join["OMCost"] + df_join["TaxesInsuranceCost"]).values
    for i in range(1, life_years + 1):
        energia_mwh = df_join[f"year{i}"].values
        ingresos = energia_mwh * ppa_eur_mwh
        out[f"year{i}"] = ingresos - om_tax

    return out

# Run for all PPAs
beneficio_resultados = {f"ppa_{ppa}": calcular_beneficio(df_joined, ppa) for ppa in PPA_VALUES}

key_demo = f"ppa_{PPA_VALUES[1]}"  # for example PPA base (50 €/MWh)
print("✔ Tablas de cashflows generadas para:", list(beneficio_resultados.keys()))
beneficio_resultados[key_demo].head()

✔ Tablas de cashflows generadas para: ['ppa_35', 'ppa_50', 'ppa_65', 'ppa_100']


,City,ILR,year0,year1,year2,year3,year4,year5,year6,year7,...,year16,year17,year18,year19,year20,year21,year22,year23,year24,year25
0,Amsterdam,1.00,-3.340417e+07,1.583604e+06,1.577760e+06,1.571932e+06,1.566119e+06,1.560322e+06,1.554541e+06,1.548776e+06,...,1.497580e+06,1.491968e+06,1.486372e+06,1.480790e+06,1.475224e+06,1.469672e+06,1.464136e+06,1.458614e+06,1.453107e+06,1.447616e+06
1,Amsterdam,1.05,-3.445186e+07,1.659585e+06,1.653464e+06,1.647359e+06,1.641271e+06,1.635200e+06,1.629145e+06,1.623106e+06,...,1.569485e+06,1.563607e+06,1.557745e+06,1.551899e+06,1.546069e+06,1.540254e+06,1.534456e+06,1.528672e+06,1.522905e+06,1.517153e+06
2,Amsterdam,1.10,-3.549513e+07,1.735641e+06,1.729244e+06,1.722863e+06,1.716500e+06,1.710154e+06,1.703825e+06,1.697513e+06,...,1.641467e+06,1.635324e+06,1.629197e+06,1.623086e+06,1.616992e+06,1.610915e+06,1.604854e+06,1.598809e+06,1.592781e+06,1.586769e+06
3,Amsterdam,1.14,-3.653840e+07,1.811662e+06,1.804988e+06,1.798331e+06,1.791693e+06,1.785072e+06,1.778469e+06,1.771884e+06,...,1.713415e+06,1.707005e+06,1.700614e+06,1.694239e+06,1.687881e+06,1.681541e+06,1.675218e+06,1.668912e+06,1.662623e+06,1.656351e+06
4,Amsterdam,1.19,-3.758167e+07,1.887515e+06,1.880564e+06,1.873632e+06,1.866719e+06,1.859824e+06,1.852948e+06,1.846091e+06,...,1.785201e+06,1.778526e+06,1.771870e+06,1.765231e+06,1.758611e+06,1.752008e+06,1.745423e+06,1.738856e+06,1.732307e+06,1.725775e+06


In [29]:
# === Discount rate by country + City→Country mapping. ===
import numpy as np
import pandas as pd

# 1) Mapping City → Country (capital cities UE + UK)
city_to_country = {
    "Madrid":"Spain","Paris":"France","Berlin":"Germany","Rome":"Italy",
    "Vienna":"Austria","Brussels":"Belgium","Sofia":"Bulgaria",
    "Zagreb":"Croatia","Nicosia":"Cyprus","Prague":"Czech Republic",
    "Copenhagen":"Denmark","Tallinn":"Estonia","Helsinki":"Finland",
    "Athens":"Greece","Budapest":"Hungary","Dublin":"Ireland",
    "Riga":"Latvia","Vilnius":"Lithuania","Luxembourg":"Luxembourg",
    "Valletta":"Malta","Amsterdam":"Netherlands","Warsaw":"Poland",
    "Lisbon":"Portugal","Bucharest":"Romania","Bratislava":"Slovakia",
    "Ljubljana":"Slovenia","Stockholm":"Sweden","London":"United Kingdom",
}

# 2) Discount rates AFTER-TAX (as decimals)
discount_rate_by_country = {
    "Austria": 0.0602,
    "Belgium": 0.0592,
    "Bulgaria": 0.0800,
    "Croatia": 0.0636,
    "Cyprus": 0.0754,
    "Czech Republic": 0.0729,
    "Denmark": 0.0457,
    "Estonia": 0.0530,
    "Finland": 0.0584,
    "France": 0.0535,
    "Germany": 0.0462,
    "Greece": 0.0757,
    "Hungary": 0.0826,
    "Ireland": 0.0506,
    "Italy": 0.0694,
    "Latvia": 0.0587,
    "Lithuania": 0.0643,
    "Luxembourg": 0.0449,
    "Malta": 0.0620,
    "Netherlands": 0.0492,
    "Poland": 0.0794,
    "Portugal": 0.0588,
    "Romania": 0.1045,
    "Slovakia": 0.0599,
    "Slovenia": 0.0615,
    "Spain": 0.0627,
    "Sweden": 0.0573,
    "United Kingdom": 0.0642,
}

# 3) Dictionary mapping City → discount rate.
def build_discount_rate_by_city(cities: list[str]) -> dict[str, float]:
    rates = {}
    for city in cities:
        country = city_to_country.get(city)
        rate = discount_rate_by_country.get(country, np.nan)
        rates[city] = rate
    return rates

# Available cities (from the energy table created earlier)
# 'full_energy' comes from the revenue block (City, ILR, AC_energy_MWh, year1..year25)
if "full_energy" not in globals():
    FULL_TABLE = "all_cities_all_ILR_energy.csv"
    full_energy = pd.read_csv(FULL_TABLE, usecols=["City","ILR","AC_energy_MWh"])
    full_energy = (full_energy.drop_duplicates(subset=["City","ILR"])
                              .sort_values(["City","ILR"])
                              .reset_index(drop=True))

cities_available = full_energy["City"].unique().tolist()
discount_rate_by_city = build_discount_rate_by_city(cities_available)

# Checks missing data
missing = {c: r for c, r in discount_rate_by_city.items() if pd.isna(r)}
if missing:
    print("⚠️ Ciudades sin tasa (usaré 7.0% por defecto en el cálculo):", list(missing))
else:
    print("✔ Tasas de descuento asignadas a todas las ciudades.")

✔ Tasas de descuento asignadas a todas las ciudades.


In [30]:
# === case study and financial evaluation functions.===
import numpy_financial as npfin

LIFE_YEARS = 25
YEAR_COLS = [f"year{i}" for i in range(1, LIFE_YEARS + 1)]

def build_cashflows(
    df_energy_base: pd.DataFrame,
    df_costs: pd.DataFrame,
    ppa_eur_mwh: float,
    module_cost_factor: float = 1.0,
    ti_by_city: dict[str, float] | None = None,
    life_years: int = LIFE_YEARS,
) -> pd.DataFrame:
    """
    Return City×ILR cashflows with columns: year0..year25.

    year0 = –CAPEX (TotalCost including module factor)

    year1..N = cash flow after tax:
    revenue – OPEX – taxes + depreciation
    (tax is calculated on EBT = revenue – OPEX – depreciation, without tax credits)
    """
    # Check duplicates City-ILR
    e = df_energy_base[["City","ILR"] + YEAR_COLS].drop_duplicates(["City","ILR"])
    c = df_costs.drop_duplicates(["City","ILR"])

    # Join 1:1
    df = pd.merge(e, c, on=["City","ILR"], how="inner")

    # Initial capex with module cost factor
    initial = (
        df["ModuleCost"] * module_cost_factor
        + df["HV_ConnectionSubstationCost"]
        + df["MountingCost"]
        + df["AccessCost"]
        + df["InverterCost"]
    )

    # Annual OPEX is constant (same as before).
    opex = (df["OMCost"] + df["TaxesInsuranceCost"]).to_numpy(float)

    # Linear depreciation of CAPEX: base = initial cost.
    depreciation = (initial / life_years).to_numpy(float)

    out = df[["City","ILR"]].copy()
    out.insert(2, "year0", -initial.values)

    # Tax rate by city (default 25% if missing).
    def _ti(city: str) -> float:
        if ti_by_city is None:
            return 0.25
        return float(ti_by_city.get(city, 0.25))

    # After-tax annual cash flows
    cities_arr = df["City"].to_numpy(str)
    for i in range(1, life_years + 1):
        energy_mwh = df[f"year{i}"].to_numpy(float)
        revenue = energy_mwh * ppa_eur_mwh
        ebt = revenue - opex - depreciation                           # Earnings Before Tax
        taxes = np.maximum(ebt, 0.0) * np.array([_ti(c) for c in cities_arr])  
        net_income = ebt - taxes
        cash_flow = net_income + depreciation                         # Depreciation
        out[f"year{i}"] = cash_flow

    return out


def evaluate_financials(
    df_cashflows: pd.DataFrame,
    discount_rate_by_city: dict[str, float],
    default_rate: float = 0.07,
) -> pd.DataFrame:
    """
    Compute NPV, IRR, and NPV/I for each City×ILR using the city’s specific discount rate.
    """
    rows = []
    cols_cf = [f"year{i}" for i in range(0, LIFE_YEARS + 1)]
    for _, r in df_cashflows.iterrows():
        city = r["City"]
        ilr  = r["ILR"]
        cf = r[cols_cf].to_numpy(dtype=float)

        dr = discount_rate_by_city.get(city)
        if pd.isna(dr):
            dr = default_rate

        # NPV and IRR
        try:
            npv = float(npfin.npv(dr, cf))
        except Exception:
            npv = np.nan
        try:
            irr = float(npfin.irr(cf))
        except Exception:
            irr = np.nan

        init = -float(r["year0"])  #Initial investment
        rows.append({
            "City": city,
            "ILR": ilr,
            "discount_rate": dr,
            "Initial_invest": init,
            "NPV": npv,
            "IRR": irr,
            "npv_over_I": (npv / init) if init != 0 else np.nan,
        })

    return pd.DataFrame(rows).sort_values(["City","ILR"]).reset_index(drop=True)


def run_parametric_financials(
    df_energy_base: pd.DataFrame,
    df_costs: pd.DataFrame,
    ppa_values: list[float],
    module_cost_factors: list[float],
    discount_rate_by_city: dict[str, float],
    default_rate: float = 0.07,
    ti_by_city: dict[str, float] | None = None,
) -> pd.DataFrame:
    all_out = []
    for ppa in ppa_values:
        for factor in module_cost_factors:
            cf = build_cashflows(
                df_energy_base, df_costs,
                ppa_eur_mwh=ppa,
                module_cost_factor=factor,
                ti_by_city=ti_by_city,
                life_years=LIFE_YEARS,
            )
            fin = evaluate_financials(cf, discount_rate_by_city, default_rate=default_rate)
            fin["ppa_eur_mwh"] = ppa
            fin["module_factor"] = factor
            fin["module_reduction_pct"] = (1 - factor) * 100.0
            all_out.append(fin)
    return pd.concat(all_out, ignore_index=True)

In [31]:
# === Run scenarios and get the base case===
# Reuse 'full_energy' (with year1..year25) and 'df_ILR_cost_euros' from the previous block.
# Base PPA case and list of PPAs
PPA_VALUES = [35, 50, 65, 100]          # €/MWh
MODULE_FACTORS = [1.00, 0.85, 0.70, 0.55]  # 1.00 = no reduction; 0.85 = -15% in modules, etc.

# 1) For convenience: df_energy_all containing only City, ILR, year1..year25
df_energy_all = full_energy[["City","ILR"] + [f"year{i}" for i in range(1, LIFE_YEARS+1)]].copy()

#  Run all scenarios
financial_results = run_parametric_financials(
    df_energy_base=df_energy_all,
    df_costs=df_ILR_cost_euros,
    ppa_values=PPA_VALUES,
    module_cost_factors=MODULE_FACTORS,
    discount_rate_by_city=discount_rate_by_city,
    default_rate=0.07, 
    ti_by_city=ti_by_city,      
)

print("✔ Escenarios calculados:", financial_results[["ppa_eur_mwh","module_factor"]].drop_duplicates().shape[0])
financial_results.head()

df_fin_ppa_35  = financial_results.query("ppa_eur_mwh == 35 and module_factor == 1.0").drop(columns=["ppa_eur_mwh","module_factor","module_reduction_pct"])
df_fin_ppa_50  = financial_results.query("ppa_eur_mwh == 50 and module_factor == 1.0").drop(columns=["ppa_eur_mwh","module_factor","module_reduction_pct"])
df_fin_ppa_65  = financial_results.query("ppa_eur_mwh == 65 and module_factor == 1.0").drop(columns=["ppa_eur_mwh","module_factor","module_reduction_pct"])
df_fin_ppa_100 = financial_results.query("ppa_eur_mwh == 100 and module_factor == 1.0").drop(columns=["ppa_eur_mwh","module_factor","module_reduction_pct"])

# Module reductions at the base PPA (50 €/MWh), equivalent to your 'df_fin_red_*'
df_fin_red_15 = financial_results.query("ppa_eur_mwh == 50 and module_factor == 0.85")
df_fin_red_30 = financial_results.query("ppa_eur_mwh == 50 and module_factor == 0.70")
df_fin_red_45 = financial_results.query("ppa_eur_mwh == 50 and module_factor == 0.55")

✔ Escenarios calculados: 16


In [32]:
# === Export with timestamp ===
from datetime import datetime
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
financial_results.to_csv(f"financial_results_parametric_{ts}.csv", index=False)
print("💾 Exportado: financial_results_parametric_", ts)

💾 Exportado: financial_results_parametric_ 2025_10_02_19_39_10


In [33]:
df_fin_ppa_50

,City,ILR,discount_rate,Initial_invest,NPV,IRR,npv_over_I
4480,Amsterdam,1.00,0.0492,3.340417e+07,-1.239021e+07,0.007505,-0.370918
4481,Amsterdam,1.05,0.0492,3.445186e+07,-1.251070e+07,0.008500,-0.363136
4482,Amsterdam,1.10,0.0492,3.549513e+07,-1.262659e+07,0.009441,-0.355728
4483,Amsterdam,1.14,0.0492,3.653840e+07,-1.274286e+07,0.010321,-0.348753
4484,Amsterdam,1.19,0.0492,3.758167e+07,-1.286086e+07,0.011141,-0.342211
...,...,...,...,...,...,...,...
5595,Zagreb,2.65,0.0636,5.434070e+07,1.582695e+06,0.066747,0.029125
5596,Zagreb,2.70,0.0636,5.510958e+07,1.249252e+06,0.066053,0.022669
5597,Zagreb,2.75,0.0636,5.587520e+07,9.089332e+05,0.065362,0.016267
5598,Zagreb,2.79,0.0636,5.664083e+07,5.581546e+05,0.064669,0.009854


In [34]:
# === Optimal ILR per country for all scenarios ===
import pandas as pd

#Requires:
# - financial_results → output of the parametric block (City, ILR, NPV, IRR, npv_over_I, ppa_eur_mwh, module_factor, ...)
# - city_to_country   → mapping City → Country (defined earlier)

def optimal_ilr_by_country(financial_results: pd.DataFrame,
                           city_to_country: dict[str, str]) -> pd.DataFrame:
    fr = financial_results.copy()

    # Add country and cleaning
    fr["Country"] = fr["City"].map(city_to_country)
    fr = fr.dropna(subset=["Country", "npv_over_I"])

    # Scenario axes (adjust if more parameters are added).
    scenario_keys = ["ppa_eur_mwh", "module_factor"]

    # Index of the row with the highest npv_over_I for each (Country, scenario) group.
    idx = fr.groupby(["Country"] + scenario_keys)["npv_over_I"].idxmax()

    # Keep ALL columns from financial_results plus Country.
    out = fr.loc[idx].copy()

    # Reorder Columns (Country primero)
    out = out.sort_values(["Country"] + scenario_keys).reset_index(drop=True)
    cols = (["Country"] +
            [c for c in out.columns if c != "Country"])
    out = out[cols]
    return out

# Run
optimal_table = optimal_ilr_by_country(financial_results, city_to_country)

print(f"✔ Países x escenarios: {optimal_table.shape[0]} filas "
      f"(países={optimal_table['Country'].nunique()}, "
      f"escenarios={financial_results[['ppa_eur_mwh','module_factor']].drop_duplicates().shape[0]})")

# Quick look
optimal_table.head()

# (Optional) export
from datetime import datetime
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
optimal_table.to_csv(f"optimal_ilr_by_country_all_scenarios_{ts}.csv", index=False)
print("💾 Exportado:", f"optimal_ilr_by_country_all_scenarios_{ts}.csv")

✔ Países x escenarios: 448 filas (países=28, escenarios=16)
💾 Exportado: optimal_ilr_by_country_all_scenarios_2025_10_02_19_39_47.csv


In [35]:
df_opt = pd.read_csv("optimal_ilr_by_country_all_scenarios.csv")

In [36]:
df_base = (df_opt
           .query("ppa_eur_mwh == 65 and module_factor == 1.0")
           .sort_values(["Country"])
           .reset_index(drop=True))
df_base

,Country,City,ILR,discount_rate,Initial_invest,NPV,IRR,npv_over_I,ppa_eur_mwh,module_factor,module_reduction_pct
0,Austria,Vienna,1.57,0.0602,4.565897e+07,1.106619e+06,0.062775,0.024237,65,1.0,0.0
1,Belgium,Brussels,1.71,0.0592,4.979725e+07,-1.027117e+07,0.036141,-0.206260,65,1.0,0.0
2,Bulgaria,Sofia,1.61,0.0800,3.523660e+07,2.092440e+07,0.145151,0.593826,65,1.0,0.0
3,Croatia,Zagreb,1.66,0.0636,3.824309e+07,1.812716e+07,0.111320,0.473998,65,1.0,0.0
4,Cyprus,Nicosia,1.47,0.0754,3.642810e+07,2.761181e+07,0.155104,0.757981,65,1.0,0.0
5,Czech Republic,Prague,1.80,0.0729,4.114584e+07,5.372102e+06,0.087463,0.130562,65,1.0,0.0
6,Denmark,Copenhagen,1.57,0.0457,4.921125e+07,-7.928597e+06,0.029080,-0.161114,65,1.0,0.0
7,Estonia,Tallinn,1.90,0.0530,4.318651e+07,2.916374e+06,0.059833,0.067530,65,1.0,0.0
8,Finland,Helsinki,1.57,0.0584,4.399974e+07,-5.698543e+06,0.044274,-0.129513,65,1.0,0.0
9,France,Paris,1.76,0.0535,4.620221e+07,-2.510349e+06,0.047840,-0.054334,65,1.0,0.0
